In [1]:
import pandas as pd
import json
import os
import time
from tqdm import tqdm

/Users/adic/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/adic/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/var/folders/06/m0cn9grx5y326z9dy_zw0pvh0000gn/T/ipykernel_3993/975688623.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pan

In [36]:
### GET ACCESS TOKEN
import requests

def get_token():
    client_id = 'bb55f61d0c724f28bbb265fcc110b1bb'
    client_secret = '518f0ffd09724428b90254eaad8be47c'

    auth_url = 'https://accounts.spotify.com/api/token'

    data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
    }
    auth_response = requests.post(auth_url, data=data)
    access_token = auth_response.json().get('access_token')
    return access_token

In [15]:
# get files from directory

files = []
for file in os.listdir("spotify_million_playlist_dataset/data/"):
    if file.endswith(".json"):
        files.append(file)
files

['mpd.slice.549000-549999.json',
 'mpd.slice.613000-613999.json',
 'mpd.slice.115000-115999.json',
 'mpd.slice.778000-778999.json',
 'mpd.slice.290000-290999.json',
 'mpd.slice.596000-596999.json',
 'mpd.slice.324000-324999.json',
 'mpd.slice.422000-422999.json',
 'mpd.slice.974000-974999.json',
 'mpd.slice.679000-679999.json',
 'mpd.slice.7000-7999.json',
 'mpd.slice.391000-391999.json',
 'mpd.slice.497000-497999.json',
 'mpd.slice.225000-225999.json',
 'mpd.slice.523000-523999.json',
 'mpd.slice.875000-875999.json',
 'mpd.slice.448000-448999.json',
 'mpd.slice.712000-712999.json',
 'mpd.slice.193000-193999.json',
 'mpd.slice.38000-38999.json',
 'mpd.slice.695000-695999.json',
 'mpd.slice.899000-899999.json',
 'mpd.slice.721000-721999.json',
 'mpd.slice.510000-510999.json',
 'mpd.slice.846000-846999.json',
 'mpd.slice.216000-216999.json',
 'mpd.slice.411000-411999.json',
 'mpd.slice.947000-947999.json',
 'mpd.slice.317000-317999.json',
 'mpd.slice.22000-22999.json',
 'mpd.slice.794000

In [38]:
#move files into dataframe in sets of 5 uptil 100

#default
#file_slice_start = 0
#file_slice_end = 5
#file_counter = 0


#added in because I messed up and did the first two sets of 5 differently
file_slice_start = 10
file_slice_end = 15
file_counter = 1

while file_slice_end <= 100:   
    df = pd.DataFrame()
    batch_of_files = files[file_slice_start:file_slice_end]
    for file in tqdm(batch_of_files):
        rows = []
        filename = "spotify_million_playlist_dataset/data/" + file
        f = open(filename)
        data = json.load(f)
        for playlist in data['playlists']:
            for track in playlist['tracks']:
                track.pop('artist_uri', None)
                track.pop('album_uri', None)
                track.pop('duration_ms', None)
                track.pop('album_name', None)
                track['playlist_pid'] = playlist['pid']
                rows.append(track)

        df = pd.concat([df, pd.DataFrame.from_dict(rows, orient='columns')])
    
    #for each batch run api call:
    
    #get new access token 
    access_token = get_token()
    
    #get song ids
    ids = list(df['track_uri'].str.replace('spotify:track:', '').unique())
    #turn into chunks of 100 ids each
    list_chunks = generate_chunks(ids)
    #turn each chunk into a string
    list_string_chunks = generate_strings(list_chunks)
    
    #call api with list of strings
    api_response = get_attributes(list_string_chunks, access_token)
    #turn json response into a dataframe
    attribute_df = create_df_attributes(api_response)
    #join onto original dataframe, export
    format_and_export(df, attribute_df, file_counter)
    
    file_slice_start = file_slice_start + 5
    file_slice_end = file_slice_end + 5
    file_counter = file_counter + 1

api_call...:  86%|███████████████████████▎   | 929/1075 [12:09<08:07,  3.34s/it]

Error: 504 - {
  "error": {
    "status": 504,
    "message": "Service didn't reply before timeout"
  }
}
<bound method Response.json of <Response [504]>>


api_call...:  88%|███████████████████████▊   | 947/1075 [12:16<00:36,  3.46it/s]

Error: 429 - {
  "error" : {
    "status" : 429
  }
}
<bound method Response.json of <Response [429]>>
Error: 429 - {
  "error" : {
    "status" : 429
  }
}
<bound method Response.json of <Response [429]>>


api_call...:  88%|███████████████████████▊   | 949/1075 [12:16<00:25,  4.90it/s]

...

Error: 429 - {
  "error" : {
    "status" : 429
  }
}
<bound method Response.json of <Response [429]>>
Error: 429 - {
  "error" : {
    "status" : 429
  }
}
<bound method Response.json of <Response [429]>>


api_call...:  27%|███████▍                   | 300/1098 [02:35<02:10,  6.10it/s]

Error: 429 - {
  "error" : {
    "status" : 429
  }
}
<bound method Response.json of <Response [429]>>


api_call...:  27%|███████▍                   | 300/1098 [02:38<07:02,  1.89it/s]


KeyboardInterrupt: 

In [ ]:
df

In [4]:
### CHUNK CREATOR FUNCTION###
## given list of song ids, returns list of 100id lists

#counter for # of chunks
def generate_chunks(ids):
    counter = 0

    final_chunk_count = len(ids) % 100

    #chunk # to stop at because next chunk will not have 100 entries
    #will be 6829 in this case
    stop_num = (int)((len(ids) - final_chunk_count) / 100)

    #list to hold chunks
    list_chunks = []

    #chunks
    slice_start = 0
    slice_end = 100

    #iteration over total ids
    while counter < stop_num:
        #temp list to be added to main chunk list
        temp_list = ids[slice_start:slice_end]
        list_chunks.append(temp_list)
        #move indeces to alter slice
        slice_start = slice_start + 100
        slice_end = slice_end + 100
        counter = counter + 1
    list_chunks

    final_chunk = ids[slice_end:slice_end+final_chunk_count]
    list_chunks.append(final_chunk)
    
    return list_chunks

In [5]:
### CHUNK STRING CREATOR FUNCTION ###
### given list of 100id lists, returns list of *comma seperated 100songs strings*###

def generate_strings(list_chunks):
    list_string_chunks =[]

    for chunk in list_chunks:
        temp_string = ''
        for song_id in chunk:
            if song_id == chunk[0]:
                temp_string = song_id
            else:
                temp_string = temp_string + ',' + song_id
        list_string_chunks.append(temp_string)

    return list_string_chunks

# API CALL : 20 Min each

In [37]:
###CALL TO WEB API###
##given list of strings where each string is 100 song ids seperated by commas
##returns responses in json format
def get_attributes(list_string_chunks, access_token):
    responses = []

    counter = 0

    #constant parts of the 'get' message
    test_url = "https://api.spotify.com/v1/audio-features/?ids="
    test_header = {
            'Authorization': 'Bearer ' + access_token
        }

    #iteration over chunks
    for chunk_string in tqdm(list_string_chunks, desc='api_call...'):
        if counter % 100 == 0:
            time.sleep(35) 
        #cusotmized url
        to_send_url = test_url + chunk_string

        #get request
        response = requests.get(to_send_url, headers=test_header)
        if response.status_code == 200:
            # Process the response data (in JSON format)
            responses.append(response.json())
        else:
            print(f"Error: {response.status_code} - {response.text}")
            print(response.json)

        counter = counter + 1

    return responses

# FORMATTING RESPONSE and Making Dataframe

In [26]:
###CREATES DATAFRAME OF SONG ATTRIBUTES###

def create_df_attributes(responses):
    audio_feature_df = pd.DataFrame()
    for response in tqdm(responses, desc='creating attribute dataframe'):
        audio_for_chunk = response['audio_features']
        rows = []
        for entry in range(len(audio_for_chunk)):
            if audio_for_chunk[entry] is not None:
                rows.append(audio_for_chunk[entry])

        audio_feature_df = pd.concat([audio_feature_df, pd.DataFrame.from_dict(rows, orient='columns')])

    return audio_feature_df

In [27]:
### format and export ###
def format_and_export(df, attribute_df,file_counter):
    #join on key=track_uri
    joined = df.join(attribute_df.set_index('uri'), on='track_uri')
    joined.to_csv("generated_json/full_dataframe_pt" + str(file_counter))

In [43]:
df = pd.read_csv("/Users/adic/Desktop/DS3000/spotifyDS300/generated_json/file_with_attributes.csv")
df

,Unnamed: 0,pos,artist_name,track_uri,track_name,playlist_pid,danceability,energy,key,loudness,...,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,0,0,Bob Dylan,spotify:track:6QHYEZlm9wyfXfEM1vSu1P,Boots of Spanish Leather,549000,0.615,0.1140,8.0,-17.285,...,0.000004,0.1660,0.401,101.189,audio_features,6QHYEZlm9wyfXfEM1vSu1P,https://api.spotify.com/v1/tracks/6QHYEZlm9wyf...,https://api.spotify.com/v1/audio-analysis/6QHY...,277107.0,4.0
1,1,1,Bob Dylan,spotify:track:3RkQ3UwOyPqpIiIvGVewuU,Mr. Tambourine Man,549000,0.390,0.3100,5.0,-14.545,...,0.000000,0.3790,0.584,177.515,audio_features,3RkQ3UwOyPqpIiIvGVewuU,https://api.spotify.com/v1/tracks/3RkQ3UwOyPqp...,https://api.spotify.com/v1/audio-analysis/3RkQ...,330533.0,4.0
2,2,2,Loggins & Messina,spotify:track:0ju1jP0cSPJ8tmojYBEI89,Danny's Song,549000,0.505,0.2580,11.0,-12.193,...,0.000000,0.1200,0.641,141.003,audio_features,0ju1jP0cSPJ8tmojYBEI89,https://api.spotify.com/v1/tracks/0ju1jP0cSPJ8...,https://api.spotify.com/v1/audio-analysis/0ju1...,254305.0,4.0
3,3,3,Bob Dylan,spotify:track:7ny2ATvjtKszCpLpfsGnVQ,A Hard Rain's A-Gonna Fall,549000,0.567,0.1980,4.0,-18.681,...,0.000871,0.1350,0.321,92.467,audio_features,7ny2ATvjtKszCpLpfsGnVQ,https://api.spotify.com/v1/tracks/7ny2ATvjtKsz...,https://api.spotify.com/v1/audio-analysis/7ny2...,412200.0,4.0
4,4,4,Bob Dylan,spotify:track:18GiV1BaXzPVYpp9rmOg0E,Blowin' In the Wind,549000,0.380,0.0993,2.0,-20.567,...,0.000023,0.0605,0.440,174.874,audio_features,18GiV1BaXzPVYpp9rmOg0E,https://api.spotify.com/v1/tracks/18GiV1BaXzPV...,https://api.spotify.com/v1/audio-analysis/18Gi...,165427.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
666650,69289,20,Sean Paul,spotify:track:29LHe8kG3PraghUZOZYsw4,Baby Boy (feat. Beyonce),679999,0.658,0.6210,1.0,-5.725,...,0.000002,0.1420,0.817,91.267,audio_features,29LHe8kG3PraghUZOZYsw4,https://api.spotify.com/v1/tracks/29LHe8kG3Pra...,https://api.spotify.com/v1/audio-analysis/29LH...,246707.0,4.0
666651,69290,21,Shakira,spotify:track:3ZFTkvIE7kyPt6Nu3PEa7V,Hips Don't Lie,679999,0.778,0.8240,10.0,-5.892,...,0.000000,0.4050,0.758,100.024,audio_features,3ZFTkvIE7kyPt6Nu3PEa7V,https://api.spotify.com/v1/tracks/3ZFTkvIE7kyP...,https://api.spotify.com/v1/audio-analysis/3ZFT...,218093.0,4.0
666652,69291,22,Destiny's Child,spotify:track:7qtAgn9mwxygsPOsUDVRRt,Survivor,679999,0.524,0.9040,8.0,-2.071,...,0.000000,0.7760,0.655,161.188,audio_features,7qtAgn9mwxygsPOsUDVRRt,https://api.spotify.com/v1/tracks/7qtAgn9mwxyg...,https://api.spotify.com/v1/audio-analysis/7qtA...,254040.0,4.0
666653,69292,23,Destiny's Child,spotify:track:0FZvjrHpAmLKj574M4VwrF,Cater 2 U,679999,0.605,0.5840,11.0,-7.043,...,0.000000,0.2350,0.464,125.802,audio_features,0FZvjrHpAmLKj574M4VwrF,https://api.spotify.com/v1/tracks/0FZvjrHpAmLK...,https://api.spotify.com/v1/audio-analysis/0FZv...,245400.0,4.0


In [44]:
df['playlist_pid'].value_counts()

playlist_pid
324371    250
549161    250
290425    249
115141    249
324419    249
         ... 
290178      5
974335      5
290582      5
324264      5
549341      5
Name: count, Length: 10000, dtype: int64